In [1]:
import platform 
platform.python_version()

'3.8.0'

In [2]:
#!pip install --upgrade google-auth-oauthlib google-auth-httplib2

In [3]:
#!pip install --upgrade google-api-python-client

In [4]:
import getpass
API_KEY = getpass.getpass("Please enter your YouTube API key: ")

Please enter your YouTube API key:  ········


In [5]:
from googleapiclient.discovery import build

client = build('youtube', 'v3', developerKey=API_KEY)

In [6]:
request = client.search().list(
    part="snippet",
    maxResults=50,
    q="titanic",
    type="video"
)
response = request.execute()

In [7]:
ids = list()
video_metadata = list()
for item in response['items']:
    video_item = {
        "ChannelID": item['snippet']['channelId'],
        "VideoTitle": item['snippet']['title'],
        "VideoDescription": item['snippet']['description'],
        "ChannelTitle": item['snippet']['channelTitle']
    }
    if item['id']['kind'] == "youtube#video":
        item.update({
            "VideoID": item['id']['videoId']
        })
        ids.append(item['id']['videoId'])
    video_metadata.append(video_item)

In [8]:
import pandas as pd

request = client.videos().list(
    part="snippet,contentDetails,statistics",
    regionCode="IN",
    id=",".join(ids),
    maxResults=50
)
response = request.execute()

In [9]:
video_meta = list()
for video in response['items']:
    meta = {
        "VideoID": video['id'],
        "VideoTitle": video['snippet']['title'],
        "LikesCount": video['statistics'].get('likeCount', 'NA'),
        "CommentCount": video['statistics'].get('commentCount', 'NA'),
        "ViewCount": video['statistics'].get('viewCount', 'NA')
    }
    video_meta.append(meta)

In [10]:
video_meta_df = pd.DataFrame.from_dict(video_meta)

In [11]:
video_meta_df = video_meta_df[video_meta_df["LikesCount"] != "NA"]
video_meta_df["LikesCount"] = video_meta_df["LikesCount"].astype(int)
sorted_video_meta_df = video_meta_df.sort_values(by=["LikesCount"], ascending=False)
top_video = sorted_video_meta_df.iloc[0]

In [12]:
metas = list()
next_page_token = None
video_id = top_video['VideoID']
comments_limit = 100
loop_count = 0

In [13]:
while loop_count < comments_limit:
    comments = client.commentThreads().list(
        part="snippet",
        videoId=video_id,
        pageToken=next_page_token
    ).execute()
    
    metas.append(comments)
    next_page_token = comments.get('nextPageToken')
    loop_count += len(comments['items'])
    if not next_page_token:
        break

In [14]:
all_comments = list()
for meta in metas:
    for item in meta['items']:
        comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
        all_comments.append(comment)
print(all_comments)

['Nah i spotted the ocean gate in that forst shot 💀', 'Why is this specific ship so important?', '😢😢😢', '&quot;Me:there was an tinitac lost in the the ocean.... &quot;<br>&quot;My dad:that exist&quot;', 'R.I.P.✝️', 'Why did the Titanic sink?<br><br><br> The immediate cause of RMS Titanic&#39;s demise was\xa0a collision with an iceberg\xa0that caused the ocean liner to sink on April 14–15, 1912. While the ship could reportedly stay afloat if as many as 4 of its 16 compartments were breached, the impact had affected at least 5 compartments.May 20, 2024', 'No. No.. this can&#39;t be noooooooooo!!!!', 'Lies. They said it will still be there in many many years.', 'Britannic will still live loger then titanic', 'Bro ther is a secert about titanic  like Olipic is titanic   titanic is Olipic  they switch places', 'why is that actually scary', 'RIP TITANIC', 'wtf   that shit is rustic', 'He says it shoed disappear well mabye we need it🟥⬛️⬜️🟨🔴⚫️🟡⚪️🅱️🔙🔱❕ ❤️💛🖤🤍', '<b>titanic</b> <br>Hi guys, you’r